# Pràctica 1:

Nom dels alumnes del grup: Jose Candón y Pau González

# Análisis exploratorio de datos (EDA) & uso de herramientas avanzadas de Python

El **análisis exploratorio de datos** (EDA, por sus siglas en inglés, *Exploratory Data Analysis*) es un proceso crítico en la ciencia de los datos que implica **explorar, resumir y visualizar** los datos para entender sus características principales. El objetivo principal del EDA es revelar patrones, tendencias y anomalías a partir de los datos, sin hacer suposiciones demasiado específicas o construir modelos demasiado complejos.

Aquí tenéis algunos de los elementos más importantes del análisis exploratorio de datos:

+ **Visualización de datos**: A través de gráficos y representaciones visuales, se muestran los datos para observar distribuciones, tendencias y relaciones. Esto puede incluir histogramas, gráficos de barras, diagramas de dispersión, gráficos de caja, etc.

+ **Estadísticas descriptivas**: Se calculan estadísticas como la media, la mediana, la desviación estándar y otras medidas resumen para describir las características básicas de las variables.

+ **Detección de anomalías**: Se busca identificar valores atípicos o anómalos que puedan indicar errores o situaciones especiales en los datos.

+ **Imputación de datos faltantes**: Si los datos tienen huecos o valores ausentes, se puede decidir cómo rellenar esos huecos de manera adecuada.

+ **Exploración de relaciones**: Se buscan correlaciones y relaciones entre las diferentes variables de los datos para identificar conexiones y dependencias.

+ **Agrupación de datos**: Se dividen los datos en subconjuntos basados en ciertas características para analizarlos más detenidamente.

+ **Transformación de datos**: Se pueden aplicar transformaciones a los datos para hacerlos más adecuados para análisis posteriores, como la estandarización o la normalización.

+ **Selección de características**: Si se trabaja con un conjunto de datos con muchas variables, se puede realizar una selección de características para reducir la complejidad y mejorar la eficiencia del análisis.

Python ofrece una amplia gama de herramientas y bibliotecas para realizar el análisis exploratorio de datos (EDA). Algunas de las bibliotecas más populares y útiles son las siguientes:

+ **NumPy**: NumPy es una biblioteca fundamental para la computación científica en Python. Proporciona funcionalidades para trabajar con matrices y vectores, lo cual es esencial para el análisis de datos.

+ **Pandas**: Pandas es una biblioteca muy popular para el análisis de datos que ofrece estructuras de datos como DataFrame y Series, que facilitan la manipulación y análisis de datos tabulares. Se puede utilizar Pandas para cargar, limpiar y explorar los datos.

+ **Matplotlib**: Matplotlib es una biblioteca para la visualización de datos que permite crear una amplia gama de gráficos y representaciones personalizadas. Es ideal para crear histogramas, diagramas de dispersión, gráficos de barras y muchas otras visualizaciones.

+ **SciPy**: SciPy es una biblioteca que amplía las funcionalidades de NumPy y proporciona herramientas adicionales para el análisis científico, incluidos estadísticos y métodos de optimización.

+ **Scikit-learn**: Si se está interesado en el aprendizaje automático, Scikit-learn es una biblioteca esencial que ofrece herramientas para clasificación, regresión, agrupación y otras tareas de aprendizaje automático. También contiene utilidades para la selección de características.

+ **Seaborn**: Seaborn es una biblioteca de visualización de datos basada en Matplotlib que simplifica la creación de gráficos estadísticos atractivos e informativos. Es especialmente útil para la visualización de relaciones y tendencias en los datos.


## 0. Objetivo de la práctica

El **objetivo de esta práctica** es aprender cómo realizar una exploración sobre un conjunto de datos real, que al final nos permita obtener la respuesta a varias preguntas.

En esta práctica exploraremos los datos de los trayectos de los **taxis amarillos de la ciudad de Nueva York** durante varios años.

Al finalizar esta práctica, deberéis ser capaces de responder la siguiente pregunta de forma detallada:

- **¿Cómo afectó la pandemia de la Covid al uso de los taxis en Nueva York?**

Para responder a esta pregunta tan genérica, la descompondremos en preguntas más concretas:

- ¿Qué distribución de encargos siguen los taxis y qué distancia / duración tienen?
- ¿Cuáles son las zonas donde más / menos taxis se toman?
- ¿Qué horarios son los más habituales?
- ¿Qué días de la semana y del mes se utilizan más?
- Etc.


### Instal·lació i importació de les llibreries necessàries

A més a més de les llibreries comentades, farem servir aquests mòduls de Python:

+ **PyArrow**: PyArrow és una biblioteca de Python que es fa servir per a l'intercanvi eficient de dades entre Python i altres llenguatges de programació, especialment C++ i Java. Aquesta biblioteca es desenvolupa com a part del projecte Apache Arrow, que és un projecte de codi obert dissenyat per a millorar el rendiment i la interoperabilitat de l'analítica de dades i les tecnologies relacionades. PyArrow permet la creació i la manipulació eficient de dades en forma de taules i columnes. PyArrow ofereix suport per a l'estructura de dades coneguda com a "Table", que és similar a una taula o un quadern de dades en altres llenguatges. Aquesta estructura de dades facilita l'organització i la manipulació de dades tabulars.

In [4]:
from platform import python_version
print(python_version())

3.12.7


In [ ]:
! pip install pyarrow

In [2]:
import pandas as pd
import numpy as np
import urllib.request
import zipfile
import os
from tqdm.notebook import tqdm
import pyarrow.parquet as pq

In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Descàrrega de dades

In [3]:
# Variables globals: els anys que estudiarem
YEARS = [2019, 2020, 2021]

In [1]:
# Descàrrega de les dades: Trip Record Data
# https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page

for year in tqdm(YEARS):
    if not os.path.exists(f'data/{year}'):
        os.makedirs(f'data/{year}', exist_ok=True)
        for month in tqdm(range(1, 13)):
            urllib.request.urlretrieve(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet', f'data/{year}/{month:02d}.parquet')

NameError: name 'tqdm' is not defined

## 2. Limpieza de datos

Para tener unos datos limpios y útiles, eliminaremos todos aquellos trayectos (filas del conjunto de datos) que contengan información *corrupta*. Por ejemplo, hay que verificar que no se dé ninguno de estos casos:

- Presencia de *missing data* (campos/columnas de los datos sin valor).
- La hora de recogida es posterior a la finalización del trayecto.
- Los datos se importan por mes y año. ¿Son coherentes los valores que contienen los datos?
- ¿Hay viajes con un número absurdo de pasajeros?
- ¿Hay viajes demasiado largos o demasiado cortos?
- ¿Hay pagos negativos?
- Etc.

Además, debemos considerar aspectos concretos de estos datos:

+ Puede haber viajes imposibles según la normativa:
    + [Más viajeros de los permitidos](https://www1.nyc.gov/site/tlc/passengers/passenger-frequently-asked-questions.page) o [velocidades ilegales](https://www.speed-limits.com/newyork).
    + [Límite de personas permitidas en un yellow taxicab](https://drive.google.com/file/d/1eiV7wdm7WrkRlM9bmekCRM6GY3Yq6GI2/view?usp=sharing)


De totes les columnes que tenen les dades, només cal fer servir les següents per contestar les preguntes:

- *tpep_pickup_datetime*: The date and time when the meter was engaged.
- *tpep_dropoff_datetime*: The date and time when the meter was disengaged.
- *Passenger_count*: The number of passengers in the vehicle.
- *Trip_distance*: The elapsed trip distance in miles reported by the taximeter.
- *PULocationID*: TLC Taxi Zone in which the taximeter was engaged
- *DOLocationID*: TLC Taxi Zone in which the taximeter was disengaged
- *Payment_type*: A numeric code signifying how the passenger paid for the trip.
    - 1= Credit card
    - 2= Cash
    - 3= No charge
    - 4= Dispute
    - 5= Unknown
    - 6= Voided trip
- *Fare_amount*: The time-and-distance fare calculated by the meter.
- *Total_amount*: The total amount charged to passengers.

In [25]:
# Carreguem les dades a un dataframe de pandas
# Per agilitzar els càlculs i reduir el temps de càlcul durant el desenvolupament,
# eliminem les columnes que no són útils i treballem amb una mostra
# uniforme de les dades (1 fila de cada 100).

def load_table(year, month, sampling = 100):
    """
    Funció que llegeix les dades descarregades i les converteix a un DataFrame
    """
    data = pq.read_table(f'data/{year}/{str(month).zfill(2)}.parquet').to_pandas()
    required_data = ['tpep_pickup_datetime',
                 'tpep_dropoff_datetime',
                 'passenger_count',
                 'trip_distance',
                 'PULocationID',
                 'DOLocationID',
                 'payment_type',
                 'fare_amount',
                 'total_amount']
    return data[required_data][::sampling]

# explorem una mica les dades
test = load_table(2019, 1)
test.shape

(76967, 9)

In [ ]:
test.tail(20000)

###  Exercici A

+ Un cop llegides les dades a una taula de Pandas, implementa una funció que faci el procés de neteja que consideris necessari. Per determinar què és necessari, **explora** els valors que apareixen a cada columna del dataframe de Pandas i decideix què cal fer.

> *Nota: Les columnes `Datatime` són un subtipus de dades de Pandas anomenades `datetime`.  Busca com tractar de forma eficient aquest tipus de dades en Pandas.*

In [ ]:
def clean_data(data, year, month):
    """
    Funció que neteja (una mostra de) les dades.
    """

    # Eliminamos filas con valores nulos
    data = data.dropna()

    #==============================================================

    # payment_type entre 1 y 2
    data = data[data['payment_type'].between(1, 2)]

    # pasajeros entre 1 y 5
    data = data[data['passenger_count'].between(1,5)]

    #==============================================================

    # fare_amount > total_amount (No es posible que el pago final sea menor)
    data = data[data['fare_amount'] <= data['total_amount']]

    # Evita pagos negativos y pagos nulos
    data = data[
        (data['fare_amount'] <= data['total_amount']) &
        (data['fare_amount'] > 0) &
        (data['total_amount'] > 0)
    ]

    #==============================================================

    # No acaba donde empieza, para que va alguien a coger un taxi para dar un circulo?
    data = data[data['PULocationID'] != data['DOLocationID']]

    #Zonas
    data = data[data['PULocationID'].between(1, 263)]
    data = data[data['DOLocationID'].between(1, 263)]

    #==============================================================
   
    # Filtramos las filas válidas (pickup <= dropoff)
    data = data[data['tpep_pickup_datetime'] <= data['tpep_dropoff_datetime']]

    # Año correcto
    data = data[data['tpep_pickup_datetime'].dt.year == year]
    data = data[data['tpep_dropoff_datetime'].dt.year == year] # Ya se filtra con el viaje menor de 6 h
    # mes correcto
    data = data[data['tpep_pickup_datetime'].dt.month == month]

    # Viaje menor a 6 horas, mayor a 3 min (1 km pillando tood en rojo (según gpt :))
    data = data[(data['tpep_dropoff_datetime'] - data['tpep_pickup_datetime']) <= pd.Timedelta(hours = 6)]
    data = data[(data['tpep_dropoff_datetime'] - data['tpep_pickup_datetime']) >= pd.Timedelta(minutes = 3)]
    #==============================================================

    # Eliminar exceso de velocidad (NO HA CAMBIOS)
    pickup = pd.to_datetime(data['tpep_pickup_datetime'])
    dropoff = pd.to_datetime(data['tpep_dropoff_datetime'])
    duration_h = (dropoff - pickup).dt.total_seconds() / 3600
    data = data[data['trip_distance'] / duration_h <= 65]

    # Distancia trip > 0.4
    data = data[data['trip_distance'] > 0.4]

    
    data = data[
    (data['fare_amount'] / data['trip_distance']).between(1.5, 4.5)]

    return data

test = clean_data(test, 2019,1)
test.shape

(7680, 9)

In [29]:
test.tail(10)

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,payment_type,fare_amount,total_amount
7661000,2019-01-31 23:56:12,2019-02-01 00:19:36,1,10.71,138,17,1,31.5,36.80
7662100,2019-01-31 23:21:38,2019-01-31 23:38:06,1,5.00,100,166,1,17.0,20.13
7662600,2019-01-31 23:10:07,2019-01-31 23:18:24,1,5.02,138,93,2,15.0,16.30
7663400,2019-01-31 23:34:19,2019-01-31 23:44:39,1,3.68,79,262,1,12.0,15.96
7664600,2019-01-31 23:30:32,2019-02-01 00:00:54,1,9.92,170,260,1,33.0,39.44
7665000,2019-01-31 23:19:27,2019-01-31 23:36:55,3,5.22,90,74,1,17.5,22.56
7665900,2019-01-31 23:13:29,2019-01-31 23:28:50,1,5.30,79,74,1,16.5,19.80
7666000,2019-01-31 23:34:21,2019-02-01 00:25:34,2,12.20,48,36,1,41.5,53.45
7666400,2019-01-31 23:02:26,2019-01-31 23:24:53,1,7.01,13,151,1,23.5,31.00
7666700,2019-01-31 23:49:55,2019-02-01 00:02:35,2,8.81,132,93,2,25.0,26.30


###  Exercici B

+ Escriu una funció que transformi la taula original a una nova taula, més adequada per tal de dur a terme l'exploració necessària. Podeu afegir columnes addicionals, precalcular valors, etc.

> Per exemple, podem transformar les dades per treballar amb les unitats de kilòmetres i hores per a representar les durades, distàncies i velocitats del trajecte. El motiu és, per una banda, que estem familiaritzades amb el Sistema Internacional d'Unitats (d'aquí el canvi de milles a km). I per altra, optar pels km i no pels metres degut a l'ordre de magnitud de les dades.

> Un cop tenim les dades normalitzades i enriquides hem de pensar en la neteja una altra vegada. Per exemple: Eliminar aquelles files que tinguin velocitat max excedeixi limit permès per llei https://www.speed-limits.com/newyork

In [ ]:
def post_processing(data):
    """
    Funció on implementar qualsevol tipus de postprocessament necessari.
    """
    # AQUÍ EL TEU CODI

    return data

test = post_processing(test)
test.shape

In [ ]:
test.tail()

Amb això podem crear un nou dataset (una única taula) que contingui tota la informació dels anys: 2019, 2020, 2021.

In [ ]:
df = pd.concat([clean_data(load_table(year, month), year, month)
                for year in tqdm(YEARS)
                for month in tqdm(range(1, 13), leave = False)],
                ignore_index=True, sort=False)
df = post_processing(df)
df.shape

In [ ]:
df.head()

## 3. Visualització de dades anuals

Començarem l'anàlisi de les dades amb la següent pregunta: **S'ha incrementat o ha disminuït amb la covid la quantitat de viatges anuals fets amb taxis?**

Per fer-ho, n'hi ha prou amb crear una figura de barres on es visualitzin la quantitat de viatges per any.

### Exercici C

+ Escriu una funció basada en `matplotlib` que generi aquesta gràfica.

In [ ]:
def bar_plot(df, column, xlabel, ylabel, title):
    """
    Funció que crea una figura de barres a partir del dataframe i
    la columna que conté la informació
    """
    # AQUÍ EL TEU CODI


bar_plot(df,
         'year',
         'Any',
         'Quantitat de viatges',
         'Quantitat de viatges per any')

+ Visualitza ara en tres gràfiques, el nombre de viatges per mesos de cada any, fent servir la funció anterior.

In [ ]:
# AQUÍ EL TEU CODI


+ Pregunta:
    + És el comportament que esperàveu? Explica la teva interpretació de les dades tenint en compte el que va passar al món durant aquests anys.

> Resposta


+ Visualitza **quants passatgers hi ha per taxi i per any**. Crea una figura **amb tres subfigures** (una per any) on es pugui veure el recompte de passatgers per any. Després repetiu el mateix gràfic visualitzant el % enlloc dels nombres ababsoluts (utilitzeu el paràmetre *norm = True*).

In [ ]:
def passengers_taxi_year(df, ylim, xlabel, ylabel, title, norm = False):
    """
    Funció que visualitza quants passatgers hi ha per taxi i per any
    """

    # AQUÍ EL TEU CODI

passengers_taxi_year(df,
                     (0, 60000),
                     'Nombre de passatgers',
                     'Recompte de passatgers',
                     'Recompte de passatgers per any')

In [ ]:
# AQUÍ EL TEU CODI


+ Com descriurieu de forma **qualitativa** l'efecte de la covid sobre els taxis en base a l'anàlisi fet fins ara?
+ Podeu calcular de forma **quantitativa** la magnitud dels canvis que heu detectat? Escriviu una funció que calculi una sèrie d'indicadors quantitatius (per exemple, quin canvi percentual hi ha en el nombre de viatges d'una sola persona, la mitja de passatgers per viatge, etc.).

> Respostes

In [ ]:
# AQUÍ EL TEU CODI


> Resposta:

## 4. Quantitat de viatges

Fins ara hem vist la quantitat de viatges que hi ha hagut a nivell de mesos i anys.

Ara podem estudiar l'efecte de la covid en el nombre de trajectes si **agreguem** les dades per hores, dies de la setmana, setmanes de l'any, etc.

### Exercici D

+ Escriu una única funció genèrica que visualitzi el nombre de trajectes per les agregacions de dades comentades i per cada any. Fes servir gràfics de línies discontínues (una línia per cada any) per veure'n l'evolució i marca amb una rodona o creu allà on està el valor.

In [ ]:
def visualize_trips(df, columns, title, xlabel, ylabel):
    """
    Funció que visualitza els viatges per diferents agregacions de dades
    """

    # AQUÍ EL TEU CODI


visualize_trips(df,
                ['pickup_hour', 'dropoff_hour'],
                title = 'Quantitat de viatges per hora',
                xlabel = 'Hora del dia', ylabel = 'Quantitat')

In [ ]:
visualize_trips(df,
                ['pickup_day', 'dropoff_day'],
                title = 'Quantitat de viatges per dia de la setmana',
                xlabel = 'Dia de la setmana',
                ylabel = 'Quantitat')

In [ ]:
visualize_trips(df,
                ['pickup_week', 'dropoff_week'],
                title = 'Quantitat de viatges per setmana de l\'any',
                xlabel = 'Setmana de l\'any',
                ylabel = 'Quanitat')

+ Com descriurieu l'efecte de la covid en cada cas (si és que en té)? Quin creieu que és el motiu?

> Resposta:


## 5. Relació distancia, temps i velocitat

A les dades tenim la **distància** que ha recorregut el taxi en cada viatge i de la informació temporal podem extreure també la **durada** d'aquest.

Ara explorarem com la covid va afectar les distàncies i les durades dels viatges, i també la velocitat dels taxis. També ens preguntarem si la densitat de trànsit va variar.

### Exercici E

+ Per començar, escriu una funció que visualitzi els **histogrames** de distància i durada per any.

In [ ]:
def visualize_histograms(df, column, title, xlabel, ylabel, xlim):
    """
    Funció que crea un histograma a partir de la informació que conté la columna del dataframe
    """

    # AQUÍ EL TEU CODI


visualize_histograms(df, 'trip_distance', title = 'Distància dels viatge per any',
                     xlabel = 'Distància (km)', ylabel = 'Quantitat', xlim = (-5, 80))

visualize_histograms(df, 'trip_duration', title = 'Durada dels viatges per any',
                     xlabel = 'Durada (h)', ylabel = 'Quantitat', xlim = (-1, 25) )

visualize_histograms(df, 'speed', title = 'Velocitat dels viatge per any',
                     xlabel = 'Velocitat (Km/h)', ylabel = 'Quanitat', xlim = (-1, 25) )

* Com creieu que la covid va afectar les distàncies i durades dels viatges?

* I a la velocitat dels taxis?

> Resposta:


+ Pregunta:

https://data.cityofnewyork.us/Transportation/NYC-Taxi-Zones/d3c5-ddgc

+ Utilitzant el DataFrame, escriu una funció que identifiqui les cinc ubicacions de recollida (PULocationID) més freqüents.
+ Per a aquestes cinc ubicacions de recollida, calcula el nombre mitjà de passatgers (passenger_count) per trajecte.
+ Filtra els trajectes on la ubicació de recollida i de deixada són la mateixa (PULocationID == DOLocationID). Calcula el percentatge d'aquests trajectes respecte al total de trajectes.

In [ ]:
def analyze_pickup_dropoff_locations(df):

    # AQUÍ EL TEU CODI

analyze_pickup_dropoff_locations(df)